[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/harpreetsahota204/car_dd_dataset_workshop/blob/main/06_using_3d_models.ipynb)


Note: If using in Google Colab, make sure you [install all the requirements listed here](https://github.com/harpreetsahota204/car_dd_dataset_workshop/blob/main/requirements.txt).

Which you can do by running the following:

```
# Install requirements directly from the URL
!pip install -r https://raw.githubusercontent.com/harpreetsahota204/car_dd_dataset_workshop/refs/heads/main/requirements.txt
```

You'll also want to connect to a GPU runtime.



# Using VGGT as a FiftyOne Remote Source Zoo Model


In [ ]:
import fiftyone as fo

# dataset = fo.load_dataset("cardd_from_hub")

# # or if you are in a new notebook
import fiftyone as fo
from fiftyone.utils.huggingface import load_from_hub

dataset = load_from_hub(
    "harpreetsahota/CarDD",
    name="cardd_from_hub",
    max_samples=100, # if you want to work with a subset of the dataset
    persistent=True,
    overwrite=True,
    )

## Set up the model source

First, you need to register the model source. You can do so as shown here:

In [ ]:
import fiftyone.zoo as foz

foz.register_zoo_model_source(
    "https://github.com/harpreetsahota204/vggt",
    overwrite=True
)

Next, you need to instantiate the model.

Note, be sure to install the following:

- `vggt@git+https://github.com/facebookresearch/vggt.git`
- `open3d`

In [ ]:
model = foz.load_zoo_model(
    "facebook/VGGT-1B",
    mode="crop", # you can also pass "pad",
    confidence_threshold=0.7
    )

Finally, you can apply the model to your dataset:

In [ ]:
dataset.apply_model(model, "depth_map_path")

Note, we are saving only the paths to the depth map as a dummy field. We won't have these as a part of our original dataset, instead we will create a Grouped Dataset (shown below):

In [ ]:
import fiftyone as fo
import os
from pathlib import Path

# Get filepaths from your existing dataset
filepaths = dataset.values("filepath")

# Create a new grouped dataset
grouped_dataset = fo.Dataset("cardd_3d", overwrite=True)
grouped_dataset.add_group_field("group", default="rgb")

# Process each filepath and create the group structure
samples = []
for filepath in filepaths:
    # Extract base information from the filepath
    path = Path(filepath)
    base_dir = path.parent
    base_name = path.stem
    
    # Create paths for each modality following your pattern
    rgb_path = filepath  # Original filepath (RGB)
    depth_path = os.path.join(base_dir, f"{base_name}_depth.png")  # Depth map
    threed_path = os.path.join(base_dir, f"{base_name}.fo3d")  # 3D point cloud
    
    # Create a group for these related samples
    group = fo.Group()
    
    # Create a sample for each modality with the appropriate group element
    rgb_sample = fo.Sample(filepath=rgb_path, group=group.element("rgb"))
    depth_sample = fo.Sample(filepath=depth_path, group=group.element("depth"))
    threed_sample = fo.Sample(filepath=threed_path, group=group.element("threed"))
    
    # Add samples to the list
    samples.extend([rgb_sample, depth_sample, threed_sample])

# Add all samples to the dataset
grouped_dataset.add_samples(samples)
grouped_dataset.save()

Now we can view the results in the FiftyOne App

In [ ]:
fo.launch_app(grouped_dataset)